In [1]:
using CSV, DataFrames, Plots, Random, Statistics, LinearAlgebra, JuMP, Gurobi, Tables
 

In [2]:
df=CSV.read("drive_results.csv");
len=size(df,2);
X= df[:,2:len-1];
y = df[:, len];
(xtrain, ytrain), (xtest, ytest) = IAI.split_data(:classification, X, y,seed=1,train_proportion=.5);
(xval,yval),(xtest,ytest)=IAI.split_data(:classification,xtest,ytest,seed=1,train_proportion=.5);

In [8]:
X

,yardline_100,down,ydstogo,goal_to_go
,Int64,Int64,Int64,Int64
1,58,1,10,0
2,56,3,8,0
3,56,4,8,0
4,98,1,10,0
5,94,3,6,0
6,96,4,8,0
7,43,1,10,0
8,30,1,10,0
9,22,3,2,0


In [4]:
lnr = IAI.OptimalTreeClassifier(criterion=:gini)
grid = IAI.GridSearch(lnr, Dict(
:max_depth => 1:5,
:minbucket => [10,20,50],
 ))
IAI.fit!(grid, xtrain, ytrain, xval, yval)
best_params1=IAI.get_best_params(grid)
lnr=IAI.get_learner(grid)

┌ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.
└ @ IAIBase /Users/iai/builds/InterpretableAI/SysImgBuilder/.julia/packages/IAIBase/ymcNn/src/precompile.jl:19
Training trees...  1%|▍                                 |  ETA: 0:10:50

InterruptException: InterruptException:

In [5]:
lnr = IAI.OptimalTreeClassifier(criterion=:gini,max_depth=5,minbucket=10,cp=4e-6)
IAI.fit!(lnr, xtrain, ytrain)


Training trees...100%|██████████████████████████████████| Time: 0:01:37


MethodError: MethodError: no method matching get_learner(::OptimalTrees.OptimalTreeClassifier)
Closest candidates are:
  get_learner(!Matched::IAIBase.GridSearch) at /Users/iai/builds/InterpretableAI/SysImgBuilder/.julia/packages/IAIBase/ymcNn/src/gridsearch.jl:165

In [8]:
test_acc=IAI.score(lnr,xtest,ytest,criterion=:misclassification)

0.6475046874376671

In [85]:
function get_splits(lnr)
    leaves=DataFrame()
    for node in 1:IAI.get_num_nodes(lnr)
        if IAI.is_leaf(lnr,node)
            new_node=IAI.get_parent(lnr,node)
            child_node=node
            leaf_df=DataFrame(IAI.get_classification_proba(lnr,node))
            while new_node!=1
                feat=IAI.get_split_feature(lnr,new_node)
                thresh=IAI.get_split_threshold(lnr,new_node)
                if !(Symbol(feat,"_lower") in names(leaf_df))
                    insertcols!(leaf_df,1,Symbol(feat,"_upper")=>Inf)
                    insertcols!(leaf_df,1,Symbol(feat,"_lower")=>-Inf)
                end
                if IAI.get_upper_child(lnr,new_node)==child_node
                    if leaf_df[1,Symbol(feat,"_lower")]==-Inf
                        leaf_df[:,Symbol(feat,"_lower")].=thresh
                    end
                else
                    if leaf_df[1,Symbol(feat,"_upper")]==Inf
                        leaf_df[:,Symbol(feat,"_upper")].=thresh
                    end
                end
                
                child_node=new_node
                new_node=IAI.get_parent(lnr,new_node)
            end
            insertcols!(leaf_df,1,:LEAF=>node)
            leaves=vcat(leaves,leaf_df,cols=:union)
        end
    end
    return(leaves)
end

get_splits (generic function with 1 method)

In [84]:
IAI.get_lower_child(lnr,5)

6

MethodError: MethodError: no method matching decision_path(::DataFrameRow{DataFrame,DataFrames.Index})
Closest candidates are:
  decision_path(!Matched::IAITrees.TreeLearner, !Matched::Union{AbstractDataFrame, AbstractArray{#s157,2} where #s157<:Real}) at /Users/iai/builds/InterpretableAI/SysImgBuilder/.julia/packages/IAITrees/PwNTS/src/api.jl:555
  decision_path(!Matched::IAIBase.GridSearch, !Matched::Any...; kwargs...) at /Users/iai/builds/InterpretableAI/SysImgBuilder/.julia/packages/IAITrees/PwNTS/src/api.jl:779

In [12]:
IAI.write_html("Tree1127.html",lnr)

996843

In [90]:
CSV.write("splits_nov27.csv",splits)

"splits_nov27.csv"

In [14]:
names(splits)

12-element Array{Symbol,1}:
 :LEAF              
 :down_lower        
 :down_upper        
 :yardline_100_lower
 :yardline_100_upper
 Symbol("-7")       
 Symbol("-3")       
 Symbol("0")        
 Symbol("2")        
 Symbol("7")        
 :ydstogo_lower     
 :ydstogo_upper     

In [15]:
scores=Array(names(splits)[6:10])

5-element Array{Symbol,1}:
 Symbol("-7")
 Symbol("-3")
 Symbol("0") 
 Symbol("2") 
 Symbol("7") 

In [88]:
function cost_functions(split_df)
    scores=[-7,-3,0,2,7]
    return_df=DataFrame()
    return_df[!,:BASE]=sum(i * split_df[:,Symbol(string(i))] for i in scores)
    #split_df[!,:LOWRISK]=sum(i^2 * split_df[:,Symbol(string(i))] for i in scores)+
    #    split_df[:,:BASE]-split_df[:,:BASE].^2
    #split_df[!,:HIGHRISK]=-sum(i^2 * split_df[:,Symbol(string(i))] for i in scores)+
    #    split_df[:,:BASE] + split_df[:,:BASE].^2
    return_df[!,:DOWN4LATE]=-7*split_df[:,Symbol("-7")]
    return_df[!,:ANYSCORE]=-split_df[:,Symbol("-7")]-split_df[:,Symbol("-3")]
    return_df[!,:DOWN3LATE]=-.5*split_df[:,Symbol("-3")]-.9*split_df[:,Symbol("-7")]
    return(return_df)
end

cost_functions (generic function with 1 method)

In [89]:
cost_splits=cost_functions(splits)

,BASE,DOWN4LATE,ANYSCORE,DOWN3LATE
,Float64,Float64,Float64,Float64
1,-6.20702,-5.90688,-0.945559,-0.810315
2,-5.55556,-4.9,-0.921111,-0.740556
3,-4.99752,-4.0678,-0.897054,-0.680973
4,-4.5747,-3.54142,-0.859837,-0.632286
5,-4.23422,-3.14393,-0.82401,-0.591658
6,-3.90207,-2.99214,-0.747529,-0.544744
7,-3.57143,-1.0,-1.0,-0.557143
8,-3.32031,-2.13281,-0.71875,-0.48125
9,-4.5935,-3.59485,-0.851626,-0.631233


In [92]:
splits_out=hcat(splits,cost_splits)
CSV.write("splits_w_costs1127.csv",splits_out)

"splits_w_costs1127.csv"

Data:

Play data with costs: 
"drive_results_w_costs1127.csv"

Filter by QB; those datasets are: (fill in when completed)

Things to do:

Run OPT on full drive results, with variations for each cost function

run OPT on different QB's, probably only for base cost function

Intepret results (in notebook)